In [264]:
import pandas as pd
import numpy as np
import re

In [265]:
# === 0. Đọc dữ liệu ===
df = pd.read_csv('CellphoneS.csv', encoding='utf-8')

In [266]:
# === 1. Xóa các cột không cần thiết nếu có ===
columns_to_drop = [
    'web-scraper-order', 'web-scraper-start-url',
    'link', 'link-href', 'type'
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

In [267]:
# === 2. Xử lý cột giá (price) ===
df['price'] = df['price'].astype(str).str.replace(r'[^\d]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [268]:
# Hiển thị các giá trị khác nhau trong cột 'internal_storage'
df['internal_storage'].unique()

array(['256 GB', '128 GB', nan, '64 GB', '32 GB', '8MB', '4MB', '128 MB',
       '16 GB', '512 GB', '1 TB', '256 MB'], dtype=object)

In [269]:
# === 3. Hàm chuẩn hóa dung lượng ===
def convert_storage(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip().upper()
    
    try:
        if 'GB' in val:
            return float(val.replace('GB', '').strip())
        elif 'MB' in val:
            return float(val.replace('MB', '').strip()) / 1024
        elif 'TB' in val:
            return float(val.replace('TB', '').strip()) * 1024
        else:
            return np.nan
    except:
        return np.nan

# Tạo cột mới đã chuẩn hóa
df['storage_GB'] = df['internal_storage'].apply(convert_storage)

# Xóa cột cũ
df.drop(columns=['internal_storage'], inplace=True)

In [270]:
# Hiển thị các giá trị khác nhau trong cột 'battery'
df['battery'].unique()

array(['6000 mAh', '5000mAh', '4000 mAh', '1000 mAh', '5,000 mAh',
       '5010 mAh', '5000 mAh', '4.500 mAh', 'Li-po 4500 mAh', '4.800 mAh',
       '4500mAh', 'Li-Po 5000 mAh', '4800 mAh', 'Li-Po 4500 mAh',
       '5.000 mAh', '4,805mAh', nan, 'Li-Ion 3040 mAh battery',
       '1020 mAh', '1.020 mAh\nTháo ráp', 'Pin chuẩn Li-Ion 800 mAh',
       'Li-Ion 1450 mAh, có thể tháo rời', '1450mAh', '1450 mAh',
       '2000 mAh', '1400mAh', '1800 mAh', 'Li-Po 5050 mAh', '5100 mAh',
       '5130 mAh', '6000mAh', '6500 mAh', 'Li-Po 3330 mAh', '2700 mAh',
       '5200 mAh', '5200mAh', '6.000 mAh', '7000 mAh', '4.730 mAh',
       'Li-Po 5000 mAh (Không thể tháo rời)', '5000 mAh (TYP)',
       '5000mAh (TYP)', '5500mAh', '5500 mAh', '5500mAh (TYP)',
       '5,000mAh', '4600 mAh', '5000 MAh', '5910 mAh', '5600mAh',
       '4805 mAh', '5000mAh (Typ)', 'Li-Po 4300 mAh', '5100mAh',
       '5800mAh (Typ)', '5630 mAh', 'Li-Po 6000 mAh, Fast charging 18W',
       'Li-Po 4000 mAh', '4.520 mAh', '5000mAh (

In [ ]:
# ==== 4. Chuẩn hóa hàm dữ liệu pin ===
# Hàm trích xuất giá trị mAh
def extract_battery_capacity(val):
    if pd.isna(val):
        return None
    # Chuyển hết về chữ thường và xóa khoảng trắng không cần thiết
    val = val.lower().strip()

    # Loại bỏ các ký hiệu không cần thiết
    val = val.replace(",", "").replace(".", "").replace("\n", " ")

    # Tìm số gần chữ 'mah' nhất
    match = re.search(r'(\d{3,5})\s*mah', val)
    if match:
        return int(match.group(1))

    # Nếu không có 'mah', tìm số đứng riêng có 3-5 chữ số (vẫn khả năng là dung lượng)
    match = re.search(r'(\d{3,5})', val)
    if match:
        return int(match.group(1))

    return None

In [272]:
# Áp dụng
df['battery_mAh'] = df['battery'].apply(extract_battery_capacity)
# Xóa cột battery
df.drop('battery', axis=1, inplace=True)

In [273]:
df.head()

,title,price,ratings,num_rates,screen_size,refresh_rate,chipset,GPU,sim_slot,RAM,operating_system,storage_GB,battery_mAh
0,ASUS ROG Phone 6 Mediatek,NaN,NaN,NaN,6.78 inches,NaN,MediaTek Dimensity 7000 Series,Adreno ™ 730,2 SIM (Nano-SIM),12 GB,Android 12,256.0,6000.0
1,ASUS ROG Phone 6 12GB 256GB,14490000.0,5.0/5,20.0,6.78 inches,NaN,Qualcomm ® Snapdragon ® 8+ thế hệ 1,Adreno ™ 730,2 SIM (Nano-SIM),12 GB,Android 12,256.0,6000.0
2,BENCO V91 4GB 128GB,2690000.0,5.0/5,1.0,6.56 inches,Tần số quét 90Hz,Unisoc T606,NaN,2 SIM (Nano-SIM),4GB + Mở rộng 4GB,Android 13,128.0,5000.0
3,Benco S1 Pro 8GB 256GB,4290000.0,NaN,NaN,6.8 inches,NaN,Unisoc T616,Mali-G57,2 SIM (Nano-SIM),8GB + Mở rộng 8GB,Android 13,256.0,5000.0
4,INOI A83 6GB 128GB,NaN,NaN,NaN,6.52 inches,NaN,Unisoc T606,NaN,2 SIM (Nano-SIM),6 GB,Android 12,128.0,4000.0


In [274]:
# === 5. Xử lý RAM ===
def extract_ram_in_gb(ram_str):
    if pd.isnull(ram_str):
        return None
    
    # Làm sạch chuỗi: loại bỏ dấu phẩy, khoảng trắng thừa, chuyển thành chữ thường
    cleaned_ram_str = str(ram_str).replace(',', '').strip().lower()

    # 1. Mẫu tìm kiếm để tính tổng RAM (có mở rộng) Ví dụ: "4GB + Mở rộng 4GB" sẽ bắt 4 và 4, tổng là 8.
    match_total = re.search(r'(\d+)\s*gb(?:\s*\+\s*mở\s*rộng\s*(\d+)\s*gb)?', cleaned_ram_str)
    if match_total:
        base_ram = int(match_total.group(1))
        expanded_ram = int(match_total.group(2)) if match_total.group(2) else 0
        return float(base_ram + expanded_ram) # Trả về tổng dưới dạng float (ví dụ: 8.0)

    # 2. Xử lý các trường hợp đơn giản chỉ có 'X GB' hoặc 'X MB'. Ví dụ: "4 GB" sẽ bắt 4 và 'gb'. "64MB" sẽ bắt 64 và 'mb'.
    match_simple = re.search(r'(\d+)\s*(gb|mb)?', cleaned_ram_str)
    if match_simple:
        num = int(match_simple.group(1))
        unit = match_simple.group(2)      
        if unit == 'mb':
            return float(num / 1024) # Chuyển MB sang GB
        else: # Mặc định là GB nếu không có đơn vị hoặc là 'gb'
            return float(num) # Trả về số nguyên dưới dạng float (ví dụ: 4.0)        
    return None 

df = pd.DataFrame(df)
# Áp dụng hàm chỉ cho cột 'RAM'
df['RAM_in_GB'] = df['RAM'].apply(extract_ram_in_gb)

print(df[['RAM', 'RAM_in_GB']].head(20)) # In 20 hàng đầu để thấy kết quả

                  RAM  RAM_in_GB
0               12 GB       12.0
1               12 GB       12.0
2   4GB + Mở rộng 4GB        8.0
3   8GB + Mở rộng 8GB       16.0
4                6 GB        6.0
5                 NaN        NaN
6                8 GB        8.0
7   6GB + Mở rộng 6GB       12.0
8   4GB + Mở rộng 4GB        8.0
9                4 GB        4.0
10                NaN        NaN
11               8 GB        8.0
12               8 GB        8.0
13               6 GB        6.0
14               4 GB        4.0
15              16 GB       16.0
16              12 GB       12.0
17              12 GB       12.0
18               8 GB        8.0
19              16 GB       16.0


In [275]:
# Xóa cột RAM 
df.drop('RAM', axis=1, inplace=True)

In [276]:
# === 6. Xử lý ratings: '4.5/5' → 4.5
if 'ratings' in df.columns:
    df['ratings'] = df['ratings'].astype(str).str.extract(r'([\d.]+)')
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')

In [277]:
# === 7. Xử lý refresh_rate: trích số Hz từ chuỗi
def extract_refresh_rate(x):
    match = re.search(r'(\d{2,3})\s*Hz', str(x))
    if match:
        return int(match.group(1))
    return None
if 'refresh_rate' in df.columns:
    df['refresh_rate'] = df['refresh_rate'].apply(extract_refresh_rate)

In [278]:
# === 8. Xử lý screen_size: trích kích thước inch từ chuỗi
def extract_screen_size(x):
    match = re.search(r'([\d.]+)\s*(inch|\"|\')', str(x), flags=re.IGNORECASE)
    if match:
        return float(match.group(1))
    return None
if 'screen_size' in df.columns:
    df['screen_size'] = df['screen_size'].apply(extract_screen_size)

In [279]:
df.head()

,title,price,ratings,num_rates,screen_size,refresh_rate,chipset,GPU,sim_slot,operating_system,storage_GB,battery_mAh,RAM_in_GB
0,ASUS ROG Phone 6 Mediatek,NaN,NaN,NaN,6.78,NaN,MediaTek Dimensity 7000 Series,Adreno ™ 730,2 SIM (Nano-SIM),Android 12,256.0,6000.0,12.0
1,ASUS ROG Phone 6 12GB 256GB,14490000.0,5.0,20.0,6.78,NaN,Qualcomm ® Snapdragon ® 8+ thế hệ 1,Adreno ™ 730,2 SIM (Nano-SIM),Android 12,256.0,6000.0,12.0
2,BENCO V91 4GB 128GB,2690000.0,5.0,1.0,6.56,90.0,Unisoc T606,NaN,2 SIM (Nano-SIM),Android 13,128.0,5000.0,8.0
3,Benco S1 Pro 8GB 256GB,4290000.0,NaN,NaN,6.80,NaN,Unisoc T616,Mali-G57,2 SIM (Nano-SIM),Android 13,256.0,5000.0,16.0
4,INOI A83 6GB 128GB,NaN,NaN,NaN,6.52,NaN,Unisoc T606,NaN,2 SIM (Nano-SIM),Android 12,128.0,4000.0,6.0


In [280]:
print(df.isnull().sum())


title                 0
price               105
ratings             139
num_rates           139
screen_size           7
refresh_rate        103
chipset              14
GPU                 101
sim_slot             38
operating_system     45
storage_GB           15
battery_mAh          28
RAM_in_GB            37
dtype: int64


In [281]:
# === 9. Loại bỏ dòng không có giá
df = df.dropna(subset=['price'])

In [282]:
# === 10. Điền median cho các cột số ===
numeric_cols = ['num_rates', 'RAM_in_GB', 'storage_GB', 'battery_mAh', 'refresh_rate']
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

In [283]:
# === 11. Điền mode cho các cột phân loại ===
categorical_cols = ['ratings', 'screen_size', 'chipset', 'GPU', 'sim_slot', 'operating_system']
for col in categorical_cols:
    if col in df.columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].mode()[0])

In [284]:
def simplify_os(os_name):
    os_name = os_name.lower()
    if "android" in os_name:
        return "Android"
    elif "ios" in os_name:
        return "iOS"
    elif "s30" in os_name:
        return "S30+"
    else:
        return "Khác"

df['operating_system'] = df['operating_system'].apply(simplify_os)


In [285]:
# === 12. Xuất file Excel
df.to_csv('du_lieu_da_xu_ly.csv', index=False,encoding='utf-8-sig')

In [286]:
# === 13. In kiểm tra
print("✅ Dữ liệu đã xử lý xong và lưu vào 'du_lieu_da_xu_ly.xlsx'")
print(df.head())
print("\n⚠️ Kiểm tra thiếu giá trị:")
print(df.isnull().sum())

✅ Dữ liệu đã xử lý xong và lưu vào 'du_lieu_da_xu_ly.xlsx'
                         title       price  ratings  num_rates  screen_size  \
1  ASUS ROG Phone 6 12GB 256GB  14490000.0      5.0       20.0         6.78   
2          BENCO V91 4GB 128GB   2690000.0      5.0        1.0         6.56   
3       Benco S1 Pro 8GB 256GB   4290000.0      5.0       10.0         6.80   
5                 INOI 288S 4G   1000000.0      5.0        1.0         2.00   
6      INOI Note 13s 8GB 256GB   2790000.0      5.0        2.0         6.95   

   refresh_rate                              chipset           GPU  \
1         120.0  Qualcomm ® Snapdragon ® 8+ thế hệ 1  Adreno ™ 730   
2          90.0                          Unisoc T606    Adreno 610   
3         120.0                          Unisoc T616      Mali-G57   
5         120.0                        Apple A18 Pro    Adreno 610   
6         120.0                          Unisoc T606    Adreno 610   

           sim_slot operating_system  storage